In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from surprise import Dataset, Reader, NormalPredictor, KNNBasic, KNNWithZScore, KNNWithMeans, KNNWithZScore, SVD
from surprise.model_selection import train_test_split, cross_validate
from collections import defaultdict
from surprise import accuracy

In [3]:
df_review = pd.read_csv("yelp_academic_dataset_review.csv")
df_review = pd.DataFrame(df_review)

In [4]:
df_business = pd.read_csv("yelp_academic_dataset_business.csv")
df_business = pd.DataFrame(df_business)

In [ ]:
df_user = pd.read_csv("yelp_academic_dataset_user.csv")
df_user = pd.DataFrame(df_user)

In [7]:
df_review

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5.0,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5.0,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4.0,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5.0,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [8]:
df_business=df_business[df_business['city']=='Philadelphia']
df_business = df_business[df_business["categories"].str.contains('Restaurants', na=False)]

# Filtrar as avaliações para apenas aquelas cujo 'business_id' está presente no df_business
df_merged = df_review[df_review['business_id'].isin(df_business['business_id'])]


In [9]:
#antes
# print(f'Number of users: {df_review["user_id"].nunique()}')
# print(f'Number of businesses: {df_review["business_id"].nunique()}')

In [10]:
#depois de selecionar apenas reviews de restaurantes em philadelphia
print(f'Number of users: {df_merged["user_id"].nunique()}')
print(f'Number of businesses: {df_merged["business_id"].nunique()}')

Number of users: 209513
Number of businesses: 5852


In [11]:
# Too big to be in matrix format (~300B cells)
df_pivot = df_merged.pivot_table(index='user_id', columns='business_id', values='stars', aggfunc='mean')

In [12]:
df_pivot.head(2)

business_id,-0M0b-XhtFagyLmsBtOe8w,-0PN_KFPtbnLQZEeb23XiA,-0TffRSXXIlBYVbb5AwfTg,-0eUa8TsXFFy0FCxHYmrjg,-1B9pP_CrRBJYPICE5WbRA,-1b2kNOowsPrPpBOK4lNkQ,-2-ih3mE8KPyeKVIzpBfPQ,-32hOCwsuKd04WO-HQyYFg,-3ArWZfDjfab8qVHf3WVtg,-3m_nXlyvdKAVNNmVirpGQ,...,zwTmOj4B_OVPMTMYijQiKg,zwWgOoaLY-RnFgNVuMc7GA,zwd4dyQ5ovnjVojWfAuhMw,zwgBxmrmPBF6N2uRTUYXrQ,zxRmQ_FWVowh8rlzLCSURQ,zxY4DgtXsVHihSUpsmwamg,zy7uNOvpykrq-XlmDY_wHA,zyMkbavgHASQtqVwaock9A,zz-fcqurtm77bZ_rVvo2Lw,zzyx5x0Z7xXWWvWnZFuxlQ
user_id,,,,,,,,,,,,,,,,,,,,,
---r61b7EpVPkb4UVme5tA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--0kuuLmuYBe3Rmu0Iycww,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# df_limited = df_review[df_review['user_id'].isin(df_review['user_id'].unique()[:1000]) &
#                     df_review['business_id'].isin(df_review['business_id'].unique()[:1000])]
# df_limited.reset_index(drop=True, inplace=True)
# df_limited

df_limited = df_merged[df_merged['user_id'].isin(df_merged['user_id'].unique()[:5000]) &
                    df_merged['business_id'].isin(df_merged['business_id'].unique()[:100000])]
df_limited.reset_index(drop=True, inplace=True)
# df_limited

#THIS ENDS NOT BEING USED
# top_users = df_merged['user_id'].value_counts().nlargest(1000).index  # Top  users by review count
# top_businesses = df_merged['business_id'].value_counts().nlargest(1000).index  # Top businesses by review count

# # Step 2: Filter the pivot table for only the top users and businesses
# df_limited3 = df_merged[df_merged.index.isin(top_users)]  # Filter rows (users) based on the top users
# df_limited3 = df_limited3[df_limited3.index.isin(top_users)]  # Filter rows (users) based on the top users

# # df_limited3 = df_limited3.loc[:, df_limited3.columns.isin(top_businesses)]  # Filter columns (businesses) based on top businesses

# df_limited3.reset_index(drop=True, inplace=True)

# print(df_limited3.shape)



In [14]:
# Creating dataset from the df_review
reader = Reader(rating_scale=(df_limited.stars.min(), df_limited.stars.max()))
data = Dataset.load_from_df(df_limited[['user_id', 'business_id', 'stars']], reader)

# Train/test split
trainset, testset = train_test_split(data, test_size=0.2)